# AUV 2D

In [ ]:
import sys
sys.path.append("../src/dynamics")
from pontryagin import system
from sympy import init_printing
from sympy import *
init_printing()

In [ ]:
# state
x, y, vx, vy = Matrix(symbols('x y v_x v_y', real=True))
state = Matrix([x, y, vx, vy])

# controls
ut, ux, uy = symbols('u_t', real=True, nonnegative=True), *symbols('u_x u_y', real=True)
control = Matrix([ut, ux, uy])

# constants
CD, m, A, T, rho = symbols('C_D m A T rho', real=True, nonnegative=True)

# state dynamics
dynamics = Matrix([
    vx,
    vy,
    T*ut*ux/m - rho*vx*sqrt(vx**2 + vy**2)*CD*A/2,
    T*ut*uy/m - rho*vy*sqrt(vx**2 + vy**2)*CD*A/2
])

# homotopy parameter
alpha = symbols('alpha', real=True, nonnegative=True)

# Lagrangian
lagrangian = alpha*ut + (1 - alpha)*ut**2

# equality constraints
equality = Matrix([sqrt(ux**2 + uy**2) - 1])

# inequality constraints
inequality = Matrix([ut-1, -ut, ux-1, -1-ux, uy-1, -1-uy])

In [ ]:
# instantiate the system
sys = system(state, control, dynamics, lagrangian)
sys.dfs

In [ ]:
# optimal thrust direction (solved by hand with geometric method)
utheta = -Matrix([sys.l[2], sys.l[3]])/sqrt(sys.l[2]**2 + sys.l[3]**2)
utheta.T

In [ ]:
# gradient of Hamiltonian
sys.KKT([(ux, utheta[0]), (uy, utheta[1])])
sys.KKTeqs

In [ ]:
# solving
sol = solve(sys.KKTeqs, sys.KKTvars, simplify=True, force=True)
s

In [ ]:
# now assign optimal controls
sys.uo = Matrix([[sol[ut]], utheta])
sys.uo.T

In [ ]:
# make some fortran code!
sys.codegen("../src/dynamics/auv2d", "F")

# Using fortran code

In [ ]:
import sys
sys.path.append("../src/dynamics")

In [ ]:
import auv2d